## Advanced Lane Finding with OpenCV

---
## One-time camera calibration using chessboard images

In [24]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

# calibrate camera
refimg = cv2.imread('./camera_cal/calibration1.jpg')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, refimg.shape[0:2], None, None)
print("Done calibrating !")

Done calibrating !


## Define all thresholding functions

In [25]:
def abs_sobel_thresh(img, orient='x', thresh=(0,255)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 255

    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 

    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 255

    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))

    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 255

    return binary_output

def saturation_threshold(img, thresh=(0,255)):
    # HLS
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output =  np.zeros_like(s_channel)
    binary_output[(s_channel >= thresh[0]) & (s_channel <= thresh[1])] = 255

    return binary_output

def value_threshold(img, thresh=(0,255)):
    # HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    binary_output =  np.zeros_like(v_channel)
    binary_output[(v_channel >= thresh[0]) & (v_channel <= thresh[1])] = 255

    return binary_output

## Apply distortion correction and thresholding on test images. Assess optimal thresholding combination

In [26]:
# distortion correction

images = glob.glob('./test_images/test*.jpg')

for i, fname in enumerate(images):

    img = cv2.imread(fname)
    img_undist = cv2.undistort(img, mtx, dist, None, mtx)
    out_name = './test_images/undist_test'+str(i)+'.jpg'
    cv2.imwrite(out_name,img_undist)

    img_thresh = abs_sobel_thresh(img_undist,orient='x',thresh=(20,255))
    out_name = './test_images/sobelx_test'+str(i)+'.jpg'
    cv2.imwrite(out_name,img_thresh)

    img_thresh = abs_sobel_thresh(img_undist,orient='y',thresh=(20,255))
    out_name = './test_images/sobely_test'+str(i)+'.jpg'
    cv2.imwrite(out_name,img_thresh)

    img_thresh = mag_thresh(img_undist,mag_thresh=(20,255))
    out_name = './test_images/mag_test'+str(i)+'.jpg'
    cv2.imwrite(out_name,img_thresh)

    img_thresh = dir_threshold(img_undist,thresh=(0.3,1.7))
    out_name = './test_images/dir_test'+str(i)+'.jpg'
    cv2.imwrite(out_name,img_thresh)

    img_thresh = saturation_threshold(img_undist,thresh=(100,255))
    out_name = './test_images/sat_test'+str(i)+'.jpg'
    cv2.imwrite(out_name,img_thresh)

    img_thresh = value_threshold(img_undist,thresh=(100,255))
    out_name = './test_images/val_test'+str(i)+'.jpg'
    cv2.imwrite(out_name,img_thresh)

print("Done with individual filters !")

Done with individual filters !


## Final chosen threshold combination after experimentation

In [27]:
def threshold(img):
    # detecting clear lines along x and y does better than magnitude or direction
    x_thresh = abs_sobel_thresh(img,orient='x',thresh=(20,255))
    y_thresh = abs_sobel_thresh(img,orient='y',thresh=(20,255))
    # does a good job detecting full color lines i.e. lanes
    s_thresh = saturation_threshold(img,thresh=(100,255))
    # does a good job eliminating saturated but dark patches i.e. shadows
    v_thresh = value_threshold(img,thresh=(100,255))
    final = np.zeros_like(img[:,:,0])
    final[((x_thresh==255) & (y_thresh==255)) | ((s_thresh==255) & (v_thresh==255))] = 255
    return final

for i, fname in enumerate(images):
    img = cv2.imread(fname)
    img_undist = cv2.undistort(img, mtx, dist, None, mtx)
    final = threshold(img_undist)
    out_name = './test_images/final'+str(i)+'.jpg'
    cv2.imwrite(out_name,final)

print("Done !")

Done !


## One-time calculation of Perspective transform

In [28]:
# one-time : doesn't change frame to frame
def P_transform(img):
    
    global P
    global Pinv
    
    h = img.shape[0]
    w = img.shape[1]
    size = (w,h)

    # define trapezoid
    top_height = 0.63*h
    bottom_height = 0.93*h
    top_len = 0.1*w
    bottom_len = 0.75*w

    top_left = [w/2 - top_len/2, top_height]
    top_right = [w/2 + top_len/2, top_height]
    bottom_left = [w/2 - bottom_len/2, bottom_height]
    bottom_right = [w/2 + bottom_len/2, bottom_height]

    src = np.float32([top_left, top_right, bottom_left, bottom_right])

    cv2.line(img, (int(top_left[0]), int(top_left[1])), (int(top_right[0]), int(top_right[1])), [0,0,255], 4)
    cv2.line(img, (int(top_left[0]), int(top_left[1])), (int(bottom_left[0]), int(bottom_left[1])), [0,0,255], 4)
    cv2.line(img, (int(bottom_left[0]), int(bottom_left[1])), (int(bottom_right[0]), int(bottom_right[1])), [0,0,255], 4)
    cv2.line(img, (int(bottom_right[0]), int(bottom_right[1])), (int(top_right[0]), int(top_right[1])), [0,0,255], 4)

    #plt.imshow(img)

    # define target rectangle
    side_trim = 0.25*w
    top_left = [side_trim, 0]
    top_right = [w - side_trim, 0]
    bottom_left = [side_trim, h]
    bottom_right = [w - side_trim, h]

    dst = np.float32([top_left, top_right, bottom_left, bottom_right])

    P = cv2.getPerspectiveTransform(src, dst)
    Pinv = cv2.getPerspectiveTransform(dst, src)

# apply calculated P to new images
def perspective(img):
    global P
    size = (img.shape[1],img.shape[0])
    result = cv2.warpPerspective(img, P, size, flags=cv2.INTER_LINEAR)
    return result

ref_img = cv2.imread('./test_images/final0.jpg')
P_transform(ref_img)

images = glob.glob('./test_images/final*.jpg')
for i, fname in enumerate(images):
    ref_img = cv2.imread('./test_images/final'+str(i)+'.jpg')
    result = perspective(ref_img)
    out_name = './test_images/warped'+str(i)+'.jpg'
    cv2.imwrite(out_name,result)
    
print("warp Done !")


warp Done !


## Finding centers,  and Line tracking

In [38]:
# Define a class to hold state for line detection
class Line():
    def __init__(self, fw, fh, tol, mppx, mppy, n):
        self.window_width = fw
        self.window_height = fh
        self.tolerance = tol
        self.meters_per_pixel_x = mppx
        self.meters_per_pixel_y = mppy
        self.last_N = n
        self.all_centers = []
        self.current_avg_centers = []
        self.lx = []  # x center points - left
        self.rx = []  # x center points - right
        self.yvals = [] # discrete y points
        self.ploty = [] # continuous y points
        self.lpoints = [] # continuous x points - left
        self.rpoints = [] # continuous x points - right
        
    # find centers for left & right lanes - at a given level
    # level = 0 for initial centers - closest to car
    # subsequent levels are as window moves up
    # prev_l = prev_r = 0 for level = 0
    def find_centers_level(self, img, level, prev_l, prev_r):
        h = img.shape[0]
        w = img.shape[1]

        window_w = self.window_width
        window_h = self.window_height
        tolerance = self.tolerance

        # slightly different based on first vs. incremental
        if (level==0):
            # pixels covered by bottom half of image - should concentrate pixles vertically on lane centers
            vslice = img[int(0.5*h):,:]
            # collapse vertically
            vsum = np.sum(vslice, axis=0)
            # split in horizontal parts - for level 0, these are the 2 equal halves of the full x range
            l_start = 0
            l_end = int(w/2)
            r_start = int(w/2)
            r_end = w
        else:
            # pixels covered by window height at current level
            vslice = img[int(h - window_h*(level+1)):int(h - window_h*level),:]
            # collapse vertically
            vsum = np.sum(vslice, axis=0)
            # split in horizontal parts - for levels > 0, this is the "tolerance range" around the last found center
            l_start = int(prev_l + window_w/2 - tolerance)
            l_end = int(prev_l + window_w/2 + tolerance)
            r_start = int(prev_r + window_w/2 - tolerance)
            r_end = int(prev_r + window_w/2 + tolerance)

        l_start = max(l_start,0)
        l_end = min(l_end,w)
        r_start = max(r_start,0)
        r_end = min(r_end,w)
        
        # now convolve to find location of maximum concentration of pixels
        conv_filter = np.ones(window_w)
        
        #print("convolving for left (%d to %d)" % (l_start, l_end))
        conv_l = np.convolve(conv_filter, vsum[l_start:l_end])
        new_l = prev_l + np.argmax(conv_l) - window_w/2
        #print("convolving for right (%d to %d)" % (r_start, r_end))
        conv_r = np.convolve(conv_filter, vsum[r_start:r_end])
        new_r = prev_r + np.argmax(conv_r) - window_w/2
        
        #print("found centers at (%d , %d)" % (new_l, new_r))

        return new_l,new_r

    # find centers for all levels
    def find_centers_all(self, img):
        centers = []        
        l = 0
        r = int(img.shape[1]/2)
        num_levels = int(img.shape[0] / self.window_height)
        
        for level in range(0,num_levels):
            new_l, new_r = self.find_centers_level(img, level, l, r)
            #print("found centers at (%d , %d)" % (new_l, new_r))
            centers.append((new_l,new_r))
            l = new_l
            r = new_r

        # track all found so far
        self.all_centers.append(centers)
        
        # avg last N - XXX not used since not maintaining state across video frames
        final = np.average(self.all_centers[-self.last_N:], axis=0)
        self.current_avg_centers = final
        #return final
    
    # fit lane lines to the centers found
    def fit_lanes(self, img):
        # collect left and right lane x points
        lx = []
        rx = []
        centers = self.current_avg_centers
        for i in range(len(centers)):
            lx.append(centers[i][0])
            rx.append(centers[i][1])

        # y points corresponding to the x centers
        h = int(img.shape[0])
        wh = self.window_height
        yvals = np.arange(h-wh/2,0,-wh)

        # fit quadratic polynomials
        left_lane = np.polyfit(yvals,lx,2)
        right_lane = np.polyfit(yvals,rx,2)

        # continuous line
        ploty = range(0,h)
        lpoints = left_lane[0]*ploty*ploty + left_lane[1]*ploty + left_lane[2]
        rpoints = right_lane[0]*ploty*ploty + right_lane[1]*ploty + right_lane[2]
        
        # save up everything with the instance
        self.lx = lx
        self.rx = rx
        self.yvals = yvals
        self.ploty = ploty
        self.lpoints = lpoints
        self.rpoints = rpoints
    
    def pipeline(self,img):
    
        global mtx
        global dist
    
        undist = cv2.undistort(img, mtx, dist, None, mtx)
    
        binary = np.zeros_like(i)
        thresh = threshold(undist)
    
        warp = perspective(thresh)
    
        self.find_centers_all(warp)
        self.fit_lanes(warp)

        res = self.draw_on_road(warp, undist)
    
        radius_l, radius_r, shift = self.curvature(warp)
        #print("(%f %f %f)" % (radius_l, radius_r, shift))

        final = annotate(res, radius_l, radius_r, shift)
    
        return final
    
    def draw_on_road(self, img, undist):

        global Pinv
    
        lpoints = self.lpoints
        rpoints = self.rpoints
        ploty = self.ploty
    
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(img).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([lpoints, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([rpoints, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Pinv, (img.shape[1], img.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
        return result
    
    def curvature(self, img):
        meters_per_pixel_x = self.meters_per_pixel_x
        meters_per_pixel_y = self.meters_per_pixel_y
        lpoints = self.lpoints
        rpoints = self.rpoints
        ploty = self.ploty

        yvals = self.yvals
        lx = instance.lx
        rx = instance.rx
    
        w = int(img.shape[1])

        calc_center = (lpoints[-1]+rpoints[-1])/2
        shift = calc_center - w/2
        real_shift = shift * meters_per_pixel_x

        real_left_lane = np.polyfit(np.array(yvals,np.float32)*meters_per_pixel_y, np.array(lx,np.float32)*meters_per_pixel_x, 2)
        real_radius_l = ((1+2*real_left_lane[0]*ploty[-1]*meters_per_pixel_y + real_left_lane[1])**2)**1.5 / np.absolute(2*real_left_lane[0])

        real_right_lane = np.polyfit(np.array(yvals,np.float32)*meters_per_pixel_y, np.array(rx,np.float32)*meters_per_pixel_x, 2)
        real_radius_r = ((1+2*real_right_lane[0]*ploty[-1]*meters_per_pixel_y + real_right_lane[1])**2)**1.5 / np.absolute(2*real_right_lane[0])
    
        return real_radius_l, real_radius_r, real_shift



## Run on test images

In [39]:
images = glob.glob('./test_images/warped*.jpg')

for i, fname in enumerate(images):
    ref_img = cv2.imread(fname)
    
    instance = Line(fw = 25, fh = 80, tol = 25, mppx=3.7/700, mppy=30/720, n=15)
    img = ref_img[:,:,0]
    instance.find_centers_all(img)
    centers = instance.current_avg_centers

    h = int(img.shape[0])
    for j in range(len(centers)):
        l = centers[j][0]
        r = centers[j][1]
        leftx = int(l - 25)
        rightx = int(l + 25)
        cv2.line(ref_img, (leftx,h), (rightx,h), [0,0,255], 10)
        cv2.line(ref_img, (rightx,h), (rightx,h-80), [0,0,255], 10)
        cv2.line(ref_img, (rightx,h-80), (leftx,h-80), [0,0,255], 10)
        cv2.line(ref_img, (leftx,h-80), (leftx,h), [0,0,255], 10)

        leftx = int(r - 25)
        rightx = int(r + 25)
        cv2.line(ref_img, (leftx,h), (rightx,h), [0,0,255], 10)
        cv2.line(ref_img, (rightx,h), (rightx,h-80), [0,0,255], 10)
        cv2.line(ref_img, (rightx,h-80), (leftx,h-80), [0,0,255], 10)
        cv2.line(ref_img, (leftx,h-80), (leftx,h), [0,0,255], 10)

        h = h-80

    out_name = './test_images/centers'+str(i)+'.jpg'
    cv2.imwrite(out_name,ref_img)

print("Find Centers test done !")
#plt.imshow(ref_img)

Find Centers test done !


## Draw on road

In [40]:
def draw_on_road_test(img, undist, instance):

    global Pinv
    
    lpoints = instance.lpoints
    rpoints = instance.rpoints
    ploty = instance.ploty
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([lpoints, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([rpoints, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Pinv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result


## Run on test images

In [41]:
images = glob.glob('./test_images/warped*.jpg')

for i, fname in enumerate(images):
    instance = Line(fw = 25, fh = 80, tol = 25, mppx=3.7/700, mppy=30/720, n=15)
    ref_img = cv2.imread(fname)
    undist = cv2.imread('./test_images/undist_test'+str(i)+'.jpg')

    img = ref_img[:,:,0]

    instance.find_centers_all(img)
    instance.fit_lanes(img)

    result = draw_on_road_test(img, undist, instance)

    out_name = './test_images/onroad'+str(i)+'.jpg'
    cv2.imwrite(out_name,result)

print("Draw on road test done !")
#plt.imshow(ref_img)

Draw on road test done !


## Find curvature and offset

In [42]:
def curvature_test(img, instance):
    meters_per_pixel_x = instance.meters_per_pixel_x
    meters_per_pixel_y = instance.meters_per_pixel_y
    lpoints = instance.lpoints
    rpoints = instance.rpoints
    ploty = instance.ploty

    yvals = instance.yvals
    lx = instance.lx
    rx = instance.rx
    
    w = int(img.shape[1])

    calc_center = (lpoints[-1]+rpoints[-1])/2
    shift = calc_center - w/2
    real_shift = shift * meters_per_pixel_x

    real_left_lane = np.polyfit(np.array(yvals,np.float32)*meters_per_pixel_y, np.array(lx,np.float32)*meters_per_pixel_x, 2)
    real_radius_l = ((1+2*real_left_lane[0]*ploty[-1]*meters_per_pixel_y + real_left_lane[1])**2)**1.5 / np.absolute(2*real_left_lane[0])

    real_right_lane = np.polyfit(np.array(yvals,np.float32)*meters_per_pixel_y, np.array(rx,np.float32)*meters_per_pixel_x, 2)
    real_radius_r = ((1+2*real_right_lane[0]*ploty[-1]*meters_per_pixel_y + real_right_lane[1])**2)**1.5 / np.absolute(2*real_right_lane[0])
    
    return real_radius_l, real_radius_r, real_shift

In [43]:
def annotate(img, radius_l, radius_r, shift):
    cv2.putText(img,"Radius left= "+str(round(radius_l,2)) , (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(img,"Radius right= "+str(round(radius_r,2)) , (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(img,"Shift = "+str(round(shift,2)) , (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    return img

## Run on test images

In [44]:
images = glob.glob('./test_images/warped*.jpg')

for i, fname in enumerate(images):
    instance = Line(fw = 25, fh = 80, tol = 25, mppx=3.7/700, mppy=30/720, n=15)
    ref_img = cv2.imread(fname)
    undist = cv2.imread('./test_images/undist_test'+str(i)+'.jpg')

    img = ref_img[:,:,0]

    instance.find_centers_all(img)
    instance.fit_lanes(img)

    result = draw_on_road_test(img, undist, instance)

    radius_l, radius_r, shift = curvature_test(img,instance)

    result = annotate(result, radius_l, radius_r, shift)

    out_name = './test_images/curvature'+str(i)+'.jpg'
    cv2.imwrite(out_name,result)

print("Curvature test done !")
#plt.imshow(ref_img)

Curvature test done !


## Complete pipeline

In [45]:
def pipeline_test(img):
    
    global mtx
    global dist
    
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    binary = np.zeros_like(i)
    thresh = threshold(undist)
    
    warp = perspective(thresh)
    
    instance = Line(fw = 25, fh = 80, tol = 25, mppx=3.7/700, mppy=30/720, n=15)    
    instance.find_centers_all(warp)
    instance.fit_lanes(warp)

    res = draw_on_road_test(warp, undist, instance)
    
    radius_l, radius_r, shift = curvature_test(warp,instance)
    #print("(%f %f %f)" % (radius_l, radius_r, shift))

    final = annotate(res, radius_l, radius_r, shift)
    
    return final

# Test
#img = cv2.imread('./test_images/test4.jpg')
#final = pipeline(img)
#plt.imshow(final)

## Process video

In [46]:
instance = Line(fw = 25, fh = 80, tol = 25, mppx=3.7/700, mppy=30/720, n=15)
clip = VideoFileClip('project_video.mp4')
outclip = clip.fl_image(lambda x: instance.pipeline(x))
outclip.write_videofile('output_video.mp4',audio=False)

[MoviePy] >>>> Building video output_video.mp4
[MoviePy] Writing video output_video.mp4


100%|█████████▉| 1260/1261 [03:31<00:00,  5.52it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video.mp4 

